<a href="https://colab.research.google.com/github/lourencocavalcante/Introduction-to-Data-Science/blob/main/CAP394_2022_Fase1_Pre_Processando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/logoinpe.png?raw=true" width = 500 align="left">](https://www.gov.br/inpe/pt-br)

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/LogoCAP.png?raw=true" width = 300 align="right">](http://www.inpe.br/posgraduacao/cap/)

# <span style="color:#336699">Análise Exploratória de Dados (EDA) para investigar ocorrência de valores não realísticos e a correlação entre os dados do disdrômetro RD80 e do radiômetro de microondas MP3000A</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 100%;">
    <b>CAP-394 – Introdução à Ciência de Dados</b>
    <br/>
    <b>Docentes:</b> Dr. Rafael Duarte Coelho dos Santos e Dr. Gilberto Ribeiro de Queiroz
    <br/><br/>
    <b>Discente:</b> Lourenço José Cavalcante Neto
    <br/>
    <b>E-mail:</b> <a href="mailto:lourenco.cavalcante@ifto.edu.b">lourenco.cavalcante@ifto.edu.br</a>

</div>

<br/>

<div style="text-align: justify;  margin-left: 20%; margin-right: 20%;">
<b>Objetivo: </b> O principal objetivo deste notebook é apresentar o Pré-processamento e análise básica de dados meteorológicos, considerando a aquisição, extração e inserção no ambiente de execução, manutenção, armazenamento, limpeza, processamento, preparação e arquitetura dos dados.
</div>

Este notebook também pode ser visualizado no diretório da disciplina CAP-394 no Github: https://github.com/lourencocavalcante/Introduction-to-Data-Science.git

# **Importação de módulos e bibliotecas**

Existem muitas bibliotecas que podem ser utilizadas para realizar o pré-processamento de dados. Entre elas podemos destacar a biblioteca **Pandas**. Trata-se de uma biblioteca para leitura, manipulação e análise de dados tabulados. Essa biblioteca oferece estruturas de dados e operações para manipular conjuntos massivos de tabelas numéricas e séries temporais de forma otimizada. No python, por convensão, as bibliotecas são importadas conforme podemos ver na célula abaixo:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import calendar
import datetime

# **Dataset**

Os dados meteorológicos são oriundos de dois equipamentos (**Radiômetro de Microondas "MP3000A"** e o **Disdrômetro "RD80"**), ambos instalados próximo à torre ATTO, em um sítio chamado Campina, na região Amazônica. O MP3000A mede diversas variáveis ambiente, entre elas a radiância no espectro de onda do microonda em diversos canais e estimativas de parâmetros de nuvens. Já o RD80 mede a distribuição das gotas de chuva que chegam à superfície.

Os dados compreendem o período de 17 de março de 2020 a 07 julho 2022.

Temos quatro tipos de arquivos:

**level1_output_sfc.txt**: 
Neste arquivo estão os dados do ambiente próximo à superfície.

**level1_output_bt.txt**: 
Neste arquivos estão os dados das medidas dos canais do microonda.

**level2_output_cld_atto.txt**: 
Neste arquivo estão as variáveis estimadas associadas a parâmetros de nuvens.

**JWD_campina_report_RZLwcDmDtNdrop_b2.1_report.txt**: 
Neste arquivo estão dados de medida da distribuição das gotas de chuva que chegam à superfície.

## **Obtendo os arquivos**

In [2]:
#Vamos baixar os dados do Google Drive para o Notebook

!gdown --id 1DflICrG3vlewOXnLYrsEqhK0WFn2Bz0U #Down. WD_campina_report_RZLwcDmDtNdrop_b2.1_report
!gdown --id 1HF3z0QjMhbOMMzvzhGLHmVHg0D6GQx5d #Down. level1_output_sfc
!gdown --id 1exlLcdlPzLOPhzC9l97TNlMnysEH0AWk #Down. level1_output_bt
!gdown --id 1suHWSYCt_3066gmPylMU9un9owYI3yVD #Down. level2_output_cld_atto

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DflICrG3vlewOXnLYrsEqhK0WFn2Bz0U
To: /content/JWD_campina_report_RZLwcDmDtNdrop_b2.1_report.txt
100% 147M/147M [00:00<00:00, 158MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1HF3z0QjMhbOMMzvzhGLHmVHg0D6GQx5d
To: /content/level1_output_sfc.txt
100% 69.7M/69.7M [00:00<00:00, 299MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  cate

Com o objetivo de facilitar a leitura dos dados e torná-los **tidy**, primeiramente será necessário carregá-los como um **dataframe**. Vamos carregar os dados e criar os dataframes:

In [3]:
#Abrindo os dados e criando os Dataframes
df_rd80_joss = pd.read_table('./JWD_campina_report_RZLwcDmDtNdrop_b2.1_report.txt',header=None, delim_whitespace=True)
df_mp3000A_1 = pd.read_table('./level1_output_sfc.txt',header=None,delim_whitespace=True)
df_mp3000A_2 = pd.read_table('./level1_output_bt.txt',header=None,delim_whitespace=True)
df_mp3000A_3 = pd.read_table('./level2_output_cld_atto.txt',header=None,delim_whitespace=True)